In [1]:
import os
import pandas as pd
import glob 
# Function to parse a file and extract metrics
def parse_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    metrics = {}
    top_k = ''

    for line in lines:
        line = line.strip()
        if line.startswith("Top"):
            top_k = line.split(' ')[1].strip()
        else:
            metric_name, metric_value = line.split(":")
            metrics[f"{metric_name}@{top_k}"] = metric_value

    return metrics


In [2]:
# Initialize an empty DataFrame to store the data
data = []
modes = ['cold_start', 'add_noise', 'missing']
# Directory path
root_dir = "./results"

# Iterate through the directory structure
for model_name in os.listdir(root_dir):
    model_dir = os.path.join(root_dir, model_name)
    for dataset in os.listdir(model_dir):
        dataset_dir = os.path.join(model_dir, dataset)
        for sub_dir in os.listdir(dataset_dir):
            for mode in modes:
                if mode in sub_dir:
                    exp_mode, percentage = mode, sub_dir.split(mode)[1].split('_')[-1]
                    file_pattern = os.path.join(dataset_dir, sub_dir, "*", "*-performance.txt")
                    # Use glob to list all files that match the pattern
                    matching_files = glob.glob(file_pattern)
                    if len(matching_files)== 0:
                        continue 
                    else:
                        if os.path.isfile(matching_files[0]):
                            metrics = parse_file(matching_files[0])
                            data.append([model_name, dataset, mode, percentage] + [
                                metrics['Hit Ratio@10'], metrics['Precision@10'], metrics['Recall@10'], metrics['NDCG@10'], 
                                metrics['Hit Ratio@20'], metrics['Precision@20'], metrics['Recall@20'], metrics['NDCG@20'], 
                                metrics['Hit Ratio@40'], metrics['Precision@40'], metrics['Recall@40'], metrics['NDCG@40'], 
                            ])

# Create a DataFrame
df = pd.DataFrame(data, columns=["Model", "Dataset", "Mode", "Percentage", "Hit Ratio@10", "Precision@10", "Recall@10", "NDCG@10",
                                  "Hit Ratio@20", "Precision@20", "Recall@20", "NDCG@20", "Hit Ratio@40", "Precision@40", "Recall@40", "NDCG@40"])

In [ ]:
df_ = df.sort_values(['Model', 'Dataset', 'Mode', 'Percentage','Hit Ratio@10'])
df_ = df_.drop_duplicates(subset=['Model', 'Dataset', 'Mode', 'Percentage'], keep='last')

In [4]:
# Save the DataFrame to a CSV file
df_.to_csv("output_final_4.csv", index=False)